# Exploration of the Mus Musculus aging dynamics
On the [Tabula Muris Senis](https://github.com/czbiohub/tabula-muris-senis) dataset.  
Data is raw RNA-seq counts from
```
https://czb-tabula-muris-senis.s3-us-west-2.amazonaws.com/Data-objects/tabula-muris-senis-facs-official-raw-obj.h5ad
```

<!-- Gene annotations are from
```
ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Mus_musculus.gene_info.gz
``` -->

<br/>

**Pre-processed** as  
- RNA counts normalized to 1.0 per cell

<br/>

Cell **statistics** (expression mean and std.dev) are calculated - with 4 levels of granularity - on subsets defined by a unique combination of  
- Animal sex
- Tissue type
- Subtissue type
- Cell type

<br/>

Expression **fold-changes** computed between subsets of the pairs of age groups are filtered as  
- Subset has >= 3 cells
<!-- - Mean expression values are >= 4 max(std. dev) apart -->
- Expression is >= 1e-12 in both age groups
- Bonferroni correction is applied to p-values by multiplying them first by the number of genes (~20k) and then by the number of subsets (2-1000 depending on granularity). Results are then thresholded by 1e-5

<br/>

### Contents
- [Part I: Exploratory analysis of a single pair of age groups on a single dataset](#Part-I:-Exploratory-analysis-of-a-single-pair-of-age-groups-on-a-single-dataset)
- [Part II: Systematic sifting through a set of age groups on both FACS and Droplet datasets](#Part-II:-Systematic-sifting-through-a-set-of-age-groups-on-both-FACS-and-Droplet-datasets)

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

import gc
import numpy as np
import pandas as pd
import scanpy as sc

# Part I: Exploratory analysis of a single pair of age groups on a single dataset

In [2]:
# FACS - Raw
data = sc.read_h5ad('../data/tabula-muris/tabula-muris-senis-facs-official-raw-obj.h5ad')
# Droplet - Raw
# data = sc.read_h5ad('../data/tabula-muris/tabula-muris-senis-droplet-official-raw-obj.h5ad')

# def load_gene_info(path):
#   """Load the NCBI Gene file and returns a {gene_name: info} dictionary"""
#   gene_info = pd.read_csv(path, delimiter='\t')
#   gene_info_map = {}
#   for _, row in gene_info.iterrows():
#     info = {
#       'type': row['type_of_gene'],
#       'desc': row['Other_designations'].split('|')[0],
#     }
#     gene_info_map[row['Symbol']] = info
#     for s in row['Synonyms'].split('|'):
#       gene_info_map[s] = info
#   return gene_info, gene_info_map

# gene_info, gene_info_map = load_gene_info('../data/Mus_musculus.gene_info')
gene_names = data.var.index.values

In [3]:
data.obs.head()

,FACS.selection,age,cell,cell_ontology_class,cell_ontology_id,free_annotation,method,mouse.id,sex,subtissue,tissue,n_genes,n_counts
index,,,,,,,,,,,,,
A10_B000497_B009023_S10.mm10-plus-0-0,Multiple,18m,A10_B000497,bulge keratinocyte,nan,nan,facs,18_53_M,male,nan,Skin,2706,1009830.0
A10_B000756_B007446_S10.mm10-plus-0-0,Viable,18m,A10_B000756,pancreatic B cell,nan,pancreatic B cell,facs,18_45_M,male,Endocrine,Pancreas,3212,978772.0
A10_B000802_B009022_S10.mm10-plus-0-0,Multiple,18m,A10_B000802,bulge keratinocyte,nan,outer bulge,facs,18_47_F,female,Skin Anagen,Skin,2243,419722.0
A10_B000927_B007456_S10.mm10-plus-0-0,"CD31+, CD45+, CD31-CD45-Sca1+, CD31-CD45-Sca1-...",18m,A10_B000927,skeletal muscle satellite cell,nan,nan,facs,18_46_F,female,Muscle Diaphragm,Limb_Muscle,2789,878950.0
A10_B001361_B007505_S10.mm10-plus-0-0,Viable,18m,A10_B001361,B cell,nan,B cell,facs,18_46_F,female,RV,Heart,1237,6587.0


In [4]:
# data.var[:100]
# data.obs.query('age in ["3m", "18m", "24m"]')
data.obs.groupby(['age', 'sex']).agg(['count'])
# np.sort(data.X[:2].toarray().ravel())[::-1][:1000]

FACS.selection   cell cell_ontology_class cell_ontology_id  \
                    count  count               count            count   
age sex                                                                 
3m  female          19315  19315               19315            19315   
    male            25203  25203               25203            25203   
18m female          19163  19163               19163            19163   
    male            14864  14864               14864            14864   
21m female            728    728                 728              728   
24m male            31551  31551               31551            31551   

           free_annotation method mouse.id subtissue tissue n_genes n_counts  
                     count  count    count     count  count   count    count  
age sex                                                                       
3m  female           19315  19315    19315     19315  19315   19315    19315  
    male             25203  25203    25203     25203  25203   25203    25203  
18m female           19163  19163    19163     19163  19163   19163    19163  
    male             14864  14864    14864     14864  14864   14864    14864  
21m female             728    728      728       728    728     728      728  
24m male             31551  31551    31551     31551  31551   31551    31551

## Split by age
As seen in the table above, top age for males and females is different.  
To avoid mixing the age groups, the analysis is done only for **3m** and **24m males**.

In [5]:
# Split the data by age groups that will be compared
def split_by_age(data, age_groups):
  data_by_age = {}
  for age in age_groups:
    idxs = data.obs['age'] == age
    data_by_age[age] = {
      'data': data[idxs],
      'obs': data.obs[idxs],
    }
  return data_by_age

age_groups = ['3m', '24m']
data_by_age = split_by_age(data, age_groups)

In [6]:
# We'll compute statistics for each subset defined by a unique set these cell attributes
subset_names = [
  'sex',
  'tissue',
  'subtissue',
  'cell_ontology_class',
]

In [7]:
# Preview cell-counts in the subsets
data_grouped = data_by_age['3m']['obs'].groupby(subset_names)
data_grouped.agg(['count']).head(20)
# data_grouped.groups

FACS.selection  \
                                                                               count   
sex    tissue        subtissue   cell_ontology_class                                   
female Aorta         Heart       aortic endothelial cell                          60   
                                 epithelial cell                                   2   
                                 macrophage                                        8   
                                 professional antigen presenting cell             17   
       BAT           Fat         B cell                                          107   
                                 NK cell                                          12   
                                 T cell                                           26   
                                 endothelial cell                                 47   
                                 epithelial cell                                   2   
                                 mesenchymal stem cell of adipose                 25   
                                 myeloid cell                                     59   
       Bladder       nan         bladder cell                                    129   
                                 bladder urothelial cell                         199   
       Brain_Myeloid Cerebellum  macrophage                                       10   
                                 microglial cell                                 303   
                     Cortex      macrophage                                        1   
                                 microglial cell                                 459   
                     Hippocampus macrophage                                        1   
                                 microglial cell                                 560   
                     Striatum    macrophage                                        1   

                                                                        age  \
                                                                      count   
sex    tissue        subtissue   cell_ontology_class                          
female Aorta         Heart       aortic endothelial cell                 60   
                                 epithelial cell                          2   
                                 macrophage                               8   
                                 professional antigen presenting cell    17   
       BAT           Fat         B cell                                 107   
                                 NK cell                                 12   
                                 T cell                                  26   
                                 endothelial cell                        47   
                                 epithelial cell                          2   
                                 mesenchymal stem cell of adipose        25   
                                 myeloid cell                            59   
       Bladder       nan         bladder cell                           129   
                                 bladder urothelial cell                199   
       Brain_Myeloid Cerebellum  macrophage                              10   
                                 microglial cell                        303   
                     Cortex      macrophage                               1   
                                 microglial cell                        459   
                     Hippocampus macrophage                               1   
                                 microglial cell                        560   
                     Striatum    macrophage                               1   

                                                                       cell  \
                                                                      count   
sex    tissue        subtissue   cell_ontology_class                 

## Split into subsets
And calculate expression **mean** and **std. dev** for each one

In [8]:
def calc_subsets(age_group, subset_names):
  """Compute mean and std. dev for all the data subsets"""
  means = {}
  stds = {}
  sample_counts = {}
  i = 0
  obs_grouped = age_group['obs'].groupby(subset_names)
  for key, group in obs_grouped:
    if not isinstance(key, tuple):
      key = (key,)
    data_array = age_group['data'][group.index].X
    # AnnData.X is usually a sparse matrix, but sometimes is an array
    if not isinstance(data_array, np.ndarray):
      data_array = data_array.toarray()
    # Skip tiny subsets
    if len(data_array.shape) < 2 or data_array.shape[0] < 3:
      continue
    data_array = data_array / (np.sum(data_array, axis=1, keepdims=True) + 1e-8)
    means[key] = np.mean(data_array, axis=0)
    stds[key] = np.std(data_array, axis=0)
    sample_counts[key] = group.index.shape[0]
    # Subsetting from AnnData can be memory-heavy, so clean it up a bit
    # (maybe, those `toarray()` calls produce the garbage?)
    if i % 10 == 0:
      gc.collect()
    i += 1
  return means, stds, sample_counts


def calc_all_subsets(data_by_age):
  means = {}
  stds = {}
  sample_counts = {}
  for age, age_group in data_by_age.items():
    means[age], stds[age], sample_counts[age] = calc_subsets(age_group, subset_names)
  return means, stds, sample_counts


means, stds, sample_counts = calc_all_subsets(data_by_age)

C:\Users\ghost\Anaconda3\lib\site-packages\anndata\core\anndata.py:846: FutureWarning: In anndata v0.7+, arrays contained within an AnnData object will maintain their dimensionality. For example, prior to v0.7 `adata[0, 0].X` returned a scalar and `adata[0, :]` returned a 1d array, post v0.7 they will return two dimensional arrays. If you would like to get a one dimensional array from your AnnData object, consider using the `adata.obs_vector`, `adata.var_vector` methods or accessing the array directly.
  warn_flatten()


## Compute fold-changes in expression

In [9]:
def calc_diffs(means, stds, sample_counts, start_age, end_age):
  """Compute expression differences between age groups"""
  diffs = {}
  confidences = {}
  for key in means[start_age].keys():
    if key not in means[end_age]:
      # No corresponding subset in the other age group - skip it     
      continue

    fold_diff = (means[end_age][key] + 1e-12) / (means[start_age][key] + 1e-12)
    with np.errstate(divide='ignore'):
      diffs[key] = np.log2(fold_diff)
    diffs[key][np.isnan(diffs[key])] = 0
    
    z_scores = np.abs(means[start_age][key] - means[end_age][key]) \
      / np.sqrt(
          np.power(stds[start_age][key] / np.sqrt(sample_counts[start_age][key]), 2) \
        + np.power(stds[end_age][key] / np.sqrt(sample_counts[end_age][key]), 2) \
      + 1e-12)
    p_values = 1.0 / np.sqrt(2.0 * np.pi) * np.exp(-0.5 * np.power(z_scores.astype(np.float64), 2))
    
    # Bonferroni correction
    p_values *= diffs[key].shape[0] * len(means)
    confidences[key] = p_values
    
    # Ignore tiny expression amounts
    min_count = 1e-12
    diffs[key][(means[start_age][key] < min_count) | (means[end_age][key] < min_count)] = 0
    
    # Zero out results with > `max_p` confidence
    max_p = 1e-5
    diffs[key][confidences[key] > max_p] = 0
    
  return diffs, confidences


diffs, confidences = calc_diffs(means, stds, sample_counts, age_groups[0], age_groups[-1])

## Output the descending-ordered fold-changes

In [10]:
def build_results(diffs, confidences, means, stds, dataset_id, start_age, end_age, gene_names, subset_names):
  """Sort the resulting differences and arrange 'em into a DataFrame with metadata"""
  # Python 3 dicts are ordered, be cool
  diff_arr = np.stack(list(diffs.values()), axis=0)
  confidence_arr = np.stack(list(confidences.values()), axis=0)
  subset_labels = [
    k if isinstance(k, tuple) else (k,) 
    for k in list(diffs.keys())
  ]
  # Descending ordered indexes into _subset x gene_ expression difference array
  ordered_diff_idxs = np.unravel_index(
    np.argsort(np.abs(diff_arr), axis=None)[::-1], 
    diff_arr.shape
  )
  gene_paths = [
    subset_labels[i] + (gene_names[j],) 
    for i, j in zip(*ordered_diff_idxs)
  ]

  # print(md_table)

  results = pd.DataFrame(
    [
      (dataset_id, start_age, end_age) \
      + path \
      + (
#         gene_info_map.get(path[-1], { 'desc': '' })['desc'],
        val,
        means[start_age][subset_labels[idx_i]][idx_j],
        means[end_age][subset_labels[idx_i]][idx_j],
        max(
          stds[start_age][subset_labels[idx_i]][idx_j], 
          stds[end_age][subset_labels[idx_i]][idx_j]
        ),
        conf,
      )
      for path, val, conf, (idx_i, idx_j) in zip(
        gene_paths, 
        diff_arr[ordered_diff_idxs], 
        confidence_arr[ordered_diff_idxs], 
        zip(*ordered_diff_idxs)
      )
    ],
    columns=[
      'dataset', 'start_age', 'end_age',
    ] \
    + subset_names \
#       'Sex', 'Tissue', 'Subtissue', 'Cell type', 'gene',  # 'Gene info',
    + [
      'gene', 
      'fold_change_log2', 
      'expression_at_start_age', 
      'expression_at_end_age', 
      'expression_max_std_dev',  # '$\\Delta$Expression'
      'p_value', 
    ]
  )
  
  # Threshold the minimum fold-change
  results = results[np.abs(results['fold_change_log2']) > 1.0]
  
  return results


def results2markdown(results):
  """Format DataFrame as a markdown table for in-Jupyter output"""
  # Generate Markdown table of the results
  nl = '\n'
  md_table = \
f"""
{'| ' + ' | '.join(results.columns) + ' |'}
{'| ' + ' | '.join(['---' for _ in np.arange(len(results.columns))]) + ' |'}
{nl.join([
  '| ' + ' | '.join(['{0}'.format(v) for v in row.values]) + ' |' 
  for _, row in results.iterrows()
])}
"""
  return md_table


results = build_results(
  diffs, confidences, means, stds, 'facs', age_groups[0], age_groups[-1], gene_names, subset_names
)
md_table = results2markdown(results.iloc[:20])
display(Markdown(md_table))


| dataset | start_age | end_age | sex | tissue | subtissue | cell_ontology_class | gene | fold_change_log2 | expression_at_start_age | expression_at_end_age | expression_max_std_dev | p_value |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Striatum | oligodendrocyte | Ppil2 | -14.671002388000488 | 9.562943159835413e-05 | 3.664892389565466e-09 | 0.00017788528930395842 | 2.0255371939722143e-15 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Striatum | oligodendrocyte | Aldh6a1 | -14.523354530334473 | 8.632671233499423e-05 | 3.664892389565466e-09 | 0.00015953877300489694 | 1.2669133046215906e-15 |
| facs | 3m | 24m | male | Aorta | Heart | aortic endothelial cell | Igfbp5 | -14.501344680786133 | 0.0008412777679041028 | 3.62733594272413e-08 | 0.0012657494517043233 | 7.773174176474623e-09 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Hippocampus | oligodendrocyte | Tbc1d15 | -14.414580345153809 | 8.843823161441833e-05 | 4.048662738398434e-09 | 0.00015793352213222533 | 3.7353678183017606e-21 |
| facs | 3m | 24m | male | Brain_Myeloid | Hippocampus | microglial cell | Hist1h3e | -14.135029792785645 | 5.3230778576107696e-05 | 2.9576592286417736e-09 | 0.00016050304111558944 | 6.097713303125484e-10 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Striatum | oligodendrocyte | Nup88 | -13.951662063598633 | 5.8082936448045075e-05 | 3.664892389565466e-09 | 0.00013402130571193993 | 1.0783409913124231e-08 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Hippocampus | oligodendrocyte | Wls | -13.895994186401367 | 0.00012345433060545474 | 8.097325476796868e-09 | 0.00019179092487320304 | 3.057292894182694e-29 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Striatum | oligodendrocyte | Pitpnb | -13.820034980773926 | 5.301817145664245e-05 | 3.664892389565466e-09 | 0.00012043784226989374 | 4.950083813425852e-09 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Cortex | oligodendrocyte | Man2b1 | -13.559788703918457 | 3.759051105589606e-05 | 3.1119700150128438e-09 | 9.843570296652615e-05 | 1.1007193347715961e-10 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Hippocampus | oligodendrocyte | Hdlbp | -13.535155296325684 | 4.80735870951321e-05 | 4.048662738398434e-09 | 0.0001034164524753578 | 3.736663043848585e-13 |
| facs | 3m | 24m | male | BAT | Fat | myeloid cell | Abpg | 13.435722351074219 | 1.832535012624703e-08 | 0.00020306331862229854 | 0.0004839695757254958 | 2.205474234643979e-11 |
| facs | 3m | 24m | male | SCAT | Fat | epithelial cell | Acta1 | 13.326435089111328 | 6.444087574664081e-08 | 0.0006619496853090823 | 0.000961235782597214 | 3.854464316014752e-06 |
| facs | 3m | 24m | male | Marrow | T-cells | precursor B cell | Akap12 | -13.285049438476562 | 0.0008065856527537107 | 8.080656499487304e-08 | 0.0007774969562888145 | 1.6993661817469358e-24 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Cortex | brain pericyte | Olfml2a | -13.226030349731445 | 0.0004391836700960994 | 4.583581869610498e-08 | 0.0004131963069085032 | 1.007301520736537e-08 |
| facs | 3m | 24m | male | BAT | Fat | myeloid cell | Car6 | 13.165288925170898 | 3.665070025249406e-08 | 0.00033669837284833193 | 0.0008942940621636808 | 1.7232921767565114e-08 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Hippocampus | oligodendrocyte | Exoc3 | -13.161014556884766 | 0.00010180020035477355 | 1.1113461262368673e-08 | 0.00015368298045359552 | 5.567368034017477e-31 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Hippocampus | oligodendrocyte | Sf3b3 | -13.023574829101562 | 3.372138962731697e-05 | 4.048662738398434e-09 | 9.410324128111824e-05 | 2.5910250445745154e-06 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Striatum | oligodendrocyte | Commd5 | -12.973742485046387 | 2.9489352527889423e-05 | 3.664892389565466e-09 | 7.0081528974697e-05 | 1.677588786114677e-07 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Hippocampus | oligodendrocyte | Atg4c | -12.929088592529297 | 6.31595030426979e-05 | 8.097325476796868e-09 | 0.00010322660091333091 | 2.25760992066934e-25 |
| facs | 3m | 24m | male | Brain_Non-Myeloid | Hippocampus | oligodendrocyte | Eif3b | -12.927616119384766 | 0.00014525896403938532 | 1.864315635202729e-08 | 0.00018231656576972455 | 1.2732793990565542e-46 |


# Part II: Systematic sifting through a set of age groups on both FACS and Droplet datasets

In [11]:
datasets = [
  {
    'id': 'facs',
    'path': '../data/tabula-muris/tabula-muris-senis-facs-official-raw-obj.h5ad',
    'age_groups': [ ('3m', '18m'), ('3m', '24m'), ('18m', '24m') ],
  },
  {
    'id': 'droplet',
    'path': '../data/tabula-muris/tabula-muris-senis-droplet-official-raw-obj.h5ad',
    'age_groups': [
      ('3m', '18m'), ('3m', '21m'), ('3m', '24m'), ('18m', '21m'), 
      ('3m', '30m'), ('21m', '24m'), ('24m', '30m'), 
    ],
  },
]

granularities = [
  [
    'sex',
    'tissue',
    'subtissue',
    'cell_ontology_class',
  ],
  [
    'sex',
    'tissue',
    'subtissue',
  ],
  [
    'sex',
    'tissue',
  ],
  [
    'sex',
  ],
]

all_results = {}
for granularity in granularities:
  granularity_results = {}
  for dataset in datasets:
    data = sc.read_h5ad(dataset['path'])
    gene_names = data.var.index.values
    for age_groups in dataset['age_groups']:
      data_by_age = split_by_age(data, age_groups)
#       subset_names = [
#         'sex',
#         'tissue',
#         'subtissue',
#         'cell_ontology_class',
#       ]
      subset_names = granularity
      # Get RNA count stats for all cell subsets defined by `subset_names`
      means, stds, sample_counts = calc_all_subsets(data_by_age)
      # Compute differences in mean expressions between age groups
      diffs, confidences = calc_diffs(means, stds, sample_counts, age_groups[0], age_groups[-1])
      if len(diffs) < 1:
        print(
          'age groups {0} in {1} dataset have no intersecting cell subsets at granularity {2}, no differences were computed' \
          .format(age_groups, dataset['id'], subset_names[-1])
        )
        continue
      results = build_results(
        diffs, confidences, means, stds, 
        dataset['id'], age_groups[0], age_groups[-1], gene_names, subset_names
      )
      results.to_csv(
        './results/{0}-{1}-{2}-{3}.csv'.format(
          dataset['id'], age_groups[0], age_groups[-1], subset_names[-1]
        )
      )
      granularity_results[(dataset['id'], age_groups[0], age_groups[-1])] = results
      all_results[(dataset['id'], age_groups[0], age_groups[-1], subset_names[-1])] = results
      
  merged_results = pd.concat(granularity_results.values()) \
    .sort_values(['fold_change_log2'], ascending=[False])
  merged_results.to_csv('./results/merged-{0}.csv'.format(subset_names[-1]))

age groups ('3m', '30m') in droplet dataset have no intersecting cell subsets at granularity cell_ontology_class, no differences were computed
age groups ('21m', '24m') in droplet dataset have no intersecting cell subsets at granularity cell_ontology_class, no differences were computed
age groups ('24m', '30m') in droplet dataset have no intersecting cell subsets at granularity cell_ontology_class, no differences were computed
age groups ('3m', '30m') in droplet dataset have no intersecting cell subsets at granularity subtissue, no differences were computed
age groups ('21m', '24m') in droplet dataset have no intersecting cell subsets at granularity subtissue, no differences were computed
age groups ('24m', '30m') in droplet dataset have no intersecting cell subsets at granularity subtissue, no differences were computed
age groups ('21m', '24m') in droplet dataset have no intersecting cell subsets at granularity tissue, no differences were computed
age groups ('21m', '24m') in droplet 

In [30]:
merged_results = pd.concat(all_results.values(), sort=False) \
  .sort_values(['subtissue', 'cell_ontology_class'], na_position='first') \
  .drop_duplicates(
    subset=['dataset', 'start_age', 'end_age', 'sex', 'tissue', 'gene', 'fold_change_log2'],
    keep='first'
  ) \
  .reset_index()

merged_results = merged_results \
  .reindex(merged_results['fold_change_log2'].abs().sort_values(ascending=[False]).index)
#   .sort_values(['fold_change_log2'], ascending=[False])

merged_results.to_csv('./results/merged.csv')
# merged_results = merged_results[merged_results['fold_change_log2'] > 1.0]
# merged_results.head(100)
# merged_grouped_results = merged_results.groupby([
#   'Sex', 'Tissue', 'Subtissue', 'Cell type', 'gene'
# ])
# merged_grouped_results.head()

# merged_results.to_csv('./results/merged.csv')

md_table = results2markdown(merged_results.iloc[:100])
display(Markdown(md_table))


| index | dataset | start_age | end_age | sex | tissue | subtissue | cell_ontology_class | gene | fold_change_log2 | expression_at_start_age | expression_at_end_age | expression_max_std_dev | p_value |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 0 | droplet | 3m | 30m | male | nan | nan | nan | Krt6a | -20.847715377807617 | 0.001424849615432322 | 7.540598701005763e-10 | 0.004652134142816067 | 0.0 |
| 0 | droplet | 24m | 30m | male | nan | nan | nan | Krt6a | -20.077863693237305 | 0.0008356425096280873 | 7.540598701005763e-10 | 0.003994071390479803 | 0.0 |
| 0 | droplet | 3m | 24m | male | Kidney | nan | nan | Slc12a1 | -18.843643188476562 | 0.0015329810557886958 | 3.257629499486825e-09 | 0.004109303466975689 | 3.82255978334385e-33 |
| 0 | facs | 18m | 24m | male | Diaphragm | Limb_Muscle | T cell | Mpo | -18.667829513549805 | 0.018219418823719025 | 4.374699003051319e-08 | 0.009715273045003414 | 2.8978210082615906e-13 |
| 1 | droplet | 3m | 30m | male | nan | nan | nan | Reg1 | 18.572494506835938 | 1.8185323247266183e-09 | 0.0007093122112564743 | 0.0037372345104813576 | 0.0 |
| 0 | droplet | 3m | 18m | female | nan | nan | nan | Ins1 | 18.413652420043945 | 9.35636457199962e-09 | 0.003267499152570963 | 0.01764858327805996 | 4.4580543198993516e-159 |
| 2 | droplet | 3m | 30m | male | nan | nan | nan | Krt13 | -18.334686279296875 | 0.0005038472008891404 | 1.5230816607925135e-09 | 0.002112005138769746 | 5.9005115922572765e-266 |
| 1 | droplet | 24m | 30m | male | nan | nan | nan | Krt13 | -18.241127014160156 | 0.00047220918349921703 | 1.5230816607925135e-09 | 0.002261705230921507 | 0.0 |
| 0 | droplet | 3m | 21m | female | nan | nan | nan | Ins1 | 18.0552921295166 | 9.35636457199962e-09 | 0.0025488133542239666 | 0.01550618838518858 | 1.0812611391687321e-206 |
| 0 | droplet | 24m | 30m | male | Kidney | nan | nan | Akr1c21 | 17.878219604492188 | 7.104028476589974e-09 | 0.0017117724055424333 | 0.0020490786992013454 | 0.0 |
| 0 | facs | 3m | 18m | female | Liver | nan | nan | Cyp3a11 | -17.80466651916504 | 0.004003104288130999 | 1.748368916310028e-08 | 0.003131460165604949 | 1.3563658113694863e-53 |
| 1 | droplet | 3m | 21m | female | nan | nan | nan | Clps | 17.79157257080078 | 8.298249865035245e-10 | 0.00018849782645702362 | 0.0027583243791013956 | 1.1060300141798306e-32 |
| 0 | facs | 18m | 24m | male | BAT | nan | nan | Krt8 | 17.753686904907227 | 1.8101212750920581e-09 | 0.00040025761700235307 | 0.0011992501094937325 | 5.797106124269085e-17 |
| 1 | facs | 18m | 24m | male | BAT | nan | nan | Wfdc2 | 17.51806640625 | 1.2183406505883454e-09 | 0.0002288699324708432 | 0.0008276984444819391 | 1.589307441645952e-10 |
| 1 | droplet | 24m | 30m | male | Kidney | nan | nan | Slc12a1 | 17.482219696044922 | 3.257629499486825e-09 | 0.0005966336466372013 | 0.002478283131495118 | 2.3606444618528014e-68 |
| 0 | facs | 3m | 24m | male | BAT | nan | nan | Spt1 | 17.31181526184082 | 6.733756663379609e-09 | 0.0010957146296277642 | 0.0035212980583310127 | 2.722388209195923e-14 |
| 1 | facs | 3m | 18m | female | Liver | nan | nan | Cd7 | 17.308961868286133 | 4.369599349018927e-09 | 0.0007096719346009195 | 0.0021582881454378366 | 5.638141478540036e-09 |
| 2 | facs | 3m | 18m | female | Liver | nan | nan | C8b | -17.196439743041992 | 0.0002187495556427166 | 1.4554748517525695e-09 | 0.0003318955423310399 | 1.2542981939861734e-11 |
| 0 | droplet | 3m | 24m | male | nan | nan | nan | Acan | -17.185163497924805 | 0.000247092975769192 | 1.6571001237863925e-09 | 0.0007695002132095397 | 0.0 |
| 1 | droplet | 3m | 24m | male | Kidney | nan | nan | Akr1c21 | -17.09027099609375 | 0.0009914026595652103 | 7.104028476589974e-09 | 0.0014598675770685077 | 7.969704766058269e-118 |
| 2 | facs | 18m | 24m | male | Heart | LA | nan | Cd79a | 17.05069923400879 | 2.999867021458158e-09 | 0.0004073975724168122 | 0.0016760412836447358 | 1.923443593787907e-09 |
| 2 | facs | 18m | 24m | male | BAT | nan | nan | Car6 | 16.91474723815918 | 1.2555376738276891e-09 | 0.00015524654008913785 | 0.0006300217937678099 | 1.2763947036067868e-07 |
| 1 | facs | 3m | 24m | male | BAT | nan | nan | Upk1a | 16.728519439697266 | 1.3503443918594371e-09 | 0.0001467411930207163 | 0.0006037256680428982 | 2.569375267702447e-07 |
| 3 | facs | 3m | 18m | female | Liver | nan | nan | Mug-ps1 | -16.627246856689453 | 3.692524114740081e-05 | 3.6377331702475146e-10 | 6.279288936639205e-05 | 4.447299467401966e-08 |
| 4 | facs | 3m | 18m | female | Liver | nan | nan | Itgb7 | 16.500688552856445 | 6.243642935999105e-09 | 0.0005790420109406114 | 0.0012782603735104203 | 3.3213411890142276e-20 |
| 5 | facs | 3m | 18m | female | Liver | nan | nan | Ltb | 16.49839210510254 | 8.32194757549587e-09 | 0.000770527170971036 | 0.0013362427707761526 | 6.176863622806161e-35 |
| 1 | droplet | 3m | 18m | female | nan | nan | nan | Ins2 | 16.4792537689209 | 1.3472934767833067e-07 | 0.012308781035244465 | 0.06671489030122757 | 5.891358233720039e-158 |
| 2 | droplet | 3m | 24m | male | Kidney | nan | nan | Timp3 | -16.35926628112793 | 0.0014221890596672893 | 1.6916153811052936e-08 | 0.0014064733404666185 | 1.9069540292621392e-265 |
| 3 | droplet | 3m | 30m | male | nan | nan | nan | Krt16 | -16.336647033691406 | 0.0006541245384141803 | 7.902871246301402e-09 | 0.0033821172546595335 | 3.9016701951628322e-174 |
| 6 | facs | 3m | 18m | female | Liver | nan | nan | Gzmb | 16.171398162841797 | 1.1417190748375106e-08 | 0.0008427000138908625 | 0.0028103836812078953 | 7.20626039797032e-07 |
| 7 | facs | 3m | 18m | female | Liver | nan | nan | Bcl2 | 16.057771682739258 | 3.5350209426354695e-09 | 0.00024120455782394856 | 0.0006095667486079037 | 1.4701172235683227e-14 |
| 4 | droplet | 3m | 30m | male | nan | nan | nan | Sprr1b | -15.983247756958008 | 0.00014758591714780778 | 2.2772850272190226e-09 | 0.000805809919256717 | 6.778993054850206e-151 |
| 1 | droplet | 3m | 24m | male | nan | nan | nan | Bpifa1 | -15.93728256225586 | 0.0007812401163391769 | 1.244943526046427e-08 | 0.007634931709617376 | 2.5946510365804102e-46 |
| 2 | droplet | 24m | 30m | male | Kidney | nan | nan | Timp3 | 15.932404518127441 | 1.6916153811052936e-08 | 0.0010579348308965564 | 0.001245922758243978 | 0.0 |
| 5 | droplet | 3m | 30m | male | nan | nan | nan | Cela2a | 15.907881736755371 | 6.525306872617875e-08 | 0.004011966288089752 | 0.02051854319870472 | 0.0 |
| 3 | facs | 18m | 24m | male | SCAT | nan | nan | Pnlip | 15.869091033935547 | 2.1346846512670936e-09 | 0.00012782316480297595 | 0.0005989376804791391 | 2.4036029309993837e-06 |
| 2 | droplet | 3m | 21m | female | nan | nan | nan | Ins2 | 15.868315696716309 | 1.3472934767833067e-07 | 0.008059432730078697 | 0.048406943678855896 | 3.597866130834103e-212 |
| 2 | facs | 3m | 24m | male | Liver | Hepatocytes | nan | Gm11428 | 15.787020683288574 | 6.501313709605938e-09 | 0.0003676501801237464 | 0.0008898014202713966 | 1.4009067660347243e-06 |
| 2 | droplet | 24m | 30m | male | nan | nan | nan | Krt16 | -15.77771282196045 | 0.0004440221528057009 | 7.902871246301402e-09 | 0.002754871966317296 | 6.609455679340917e-208 |
| 4 | facs | 18m | 24m | male | SCAT | nan | nan | Bpifa2 | 15.724828720092773 | 9.655168220490395e-09 | 0.0005229376256465912 | 0.002381042344495654 | 5.890417360940024e-07 |
| 6 | droplet | 3m | 30m | male | nan | nan | nan | Krt24 | -15.643794059753418 | 0.0001719389547361061 | 3.3573281932319787e-09 | 0.0008327234536409378 | 3.9776988992883226e-193 |
| 3 | droplet | 24m | 30m | male | Kidney | nan | nan | Spink3 | 15.628938674926758 | 7.925944345288372e-08 | 0.00401639100164175 | 0.004533448256552219 | 0.0 |
| 8 | facs | 3m | 18m | female | Liver | nan | nan | H2-Oa | 15.625594139099121 | 2.9265192491578773e-09 | 0.0001480036007706076 | 0.0004500212089624256 | 5.9931674919056075e-09 |
| 2 | droplet | 3m | 18m | male | nan | nan | nan | Otor | -15.598246574401855 | 8.954067743616179e-05 | 1.8040132720997804e-09 | 0.0009266487904824317 | 7.79834136535557e-40 |
| 3 | droplet | 24m | 30m | male | nan | nan | nan | Reg4 | 15.590713500976562 | 6.510892380795497e-10 | 3.217950143152848e-05 | 0.0010381421307101846 | 1.4491491672295825e-07 |
| 9 | facs | 3m | 18m | female | Liver | nan | nan | Gimap7 | 15.590660095214844 | 4.321927260519942e-09 | 0.00021332137112040073 | 0.0004652552306652069 | 9.825238401932697e-21 |
| 4 | droplet | 24m | 30m | male | Kidney | nan | nan | Calml4 | 15.518572807312012 | 9.33637522848585e-09 | 0.00043830944923684 | 0.0005079015390947461 | 0.0 |
| 7 | droplet | 3m | 30m | male | nan | nan | nan | Krt76 | -15.485835075378418 | 8.998803969006985e-05 | 1.9600290279697674e-09 | 0.0013476306339725852 | 1.7644134545921768e-17 |
| 5 | facs | 18m | 24m | male | BAT | nan | nan | Serpina3k | 15.464133262634277 | 2.5699495864728306e-09 | 0.0001162148400908336 | 0.0005148763302713633 | 8.16500282506412e-06 |
| 10 | facs | 3m | 18m | female | Liver | nan | nan | Gpr18 | 15.43760871887207 | 5.390813573313835e-09 | 0.00023928642622195184 | 0.0006294572958722711 | 3.635107019059941e-13 |
| 11 | facs | 3m | 18m | female | Liver | nan | nan | Epsti1 | 15.416228294372559 | 2.6395756691499628e-09 | 0.00011546375753823668 | 0.0002778440248221159 | 2.587775571213705e-16 |
| 3 | facs | 3m | 24m | male | Liver | Hepatocytes | nan | Vsig4 | 15.409947395324707 | 2.2805055621688552e-08 | 0.0009929024381563067 | 0.0021960290614515543 | 1.3983886345776335e-08 |
| 6 | facs | 18m | 24m | male | BAT | nan | nan | Cyp3a11 | 15.404548645019531 | 1.5688114141099163e-09 | 6.808925536461174e-05 | 0.0003015478723682463 | 9.15650897137691e-06 |
| 8 | droplet | 3m | 30m | male | nan | nan | nan | Krt35 | -15.403432846069336 | 5.9787900681840256e-05 | 1.3784885455336848e-09 | 0.0006195657188072801 | 1.983344831525696e-38 |
| 2 | droplet | 3m | 24m | male | nan | nan | nan | Col10a1 | -15.397372245788574 | 0.00013596405915450305 | 3.149309701910852e-09 | 0.0011989612830802798 | 8.840318541315366e-57 |
| 12 | facs | 3m | 18m | female | Liver | nan | nan | Cytip | 15.379805564880371 | 2.5020585603385825e-09 | 0.0001067219563992694 | 0.0002847565629053861 | 1.2968911294809515e-12 |
| 3 | droplet | 3m | 18m | male | nan | nan | nan | Ins2 | 15.35195255279541 | 3.4747114341371343e-07 | 0.014531763270497322 | 0.07472457736730576 | 8.764949204679453e-184 |
| 4 | droplet | 24m | 30m | male | nan | nan | nan | Sprr1b | -15.301532745361328 | 9.200860949931666e-05 | 2.2772850272190226e-09 | 0.0006060250452719629 | 1.7629421891008577e-167 |
| 7 | facs | 18m | 24m | male | BAT | nan | nan | Ldhc | 15.265901565551758 | 3.7216283388374904e-09 | 0.00014667087816633284 | 0.0005216369754634798 | 5.636462638236408e-11 |
| 13 | facs | 3m | 18m | female | Liver | nan | nan | Mug2 | -15.216898918151855 | 0.00023767168750055134 | 6.239714522848772e-09 | 0.00026413906016387045 | 1.0574008308530658e-24 |
| 5 | droplet | 24m | 30m | male | Kidney | nan | nan | Hnf4a | 15.215368270874023 | 4.094030003898297e-09 | 0.00015578995225951076 | 0.00022968155099079013 | 0.0 |
| 14 | facs | 3m | 18m | female | Liver | nan | nan | Faim3 | 15.14059066772461 | 1.3583413505102726e-08 | 0.0004906964604742825 | 0.0014992880169302225 | 7.39712056172973e-09 |
| 3 | droplet | 3m | 24m | male | nan | nan | nan | Sftpc | -15.130048751831055 | 0.0005013013142161071 | 1.397877724684804e-08 | 0.010098690167069435 | 3.061772708965985e-08 |
| 5 | droplet | 24m | 30m | male | nan | nan | nan | Krt76 | -15.094521522521973 | 6.861007568659261e-05 | 1.9600290279697674e-09 | 0.0011496132938191295 | 7.706264693083828e-25 |
| 3 | droplet | 3m | 24m | male | Kidney | nan | nan | Hnf4a | -15.06579303741455 | 0.00014044703857507557 | 4.094030003898297e-09 | 0.00025845630443654954 | 7.582661511603972e-73 |
| 15 | facs | 3m | 18m | female | Liver | nan | nan | Map4k1 | 15.054169654846191 | 3.7320861956402496e-09 | 0.00012700608931481838 | 0.0003932522376999259 | 1.70981692634392e-08 |
| 9 | droplet | 3m | 30m | male | nan | nan | nan | Col2a1 | -15.048543930053711 | 0.0006920801242813468 | 2.042075841757196e-08 | 0.002711151260882616 | 1.352097492857714e-305 |
| 16 | facs | 3m | 18m | female | Liver | nan | nan | Ly9 | 14.989109992980957 | 3.1643150322224756e-09 | 0.00010294104868080467 | 0.00033694496960379183 | 3.2453199075171286e-07 |
| 6 | droplet | 24m | 30m | male | nan | nan | nan | Krt24 | -14.955361366271973 | 0.00010669292532838881 | 3.3573281932319787e-09 | 0.0007243055733852088 | 4.53845087578971e-162 |
| 17 | facs | 3m | 18m | female | Liver | nan | nan | Napsa | 14.937080383300781 | 9.29010735006841e-09 | 0.00029145841835998 | 0.0006998045137152076 | 1.6216104920806243e-16 |
| 4 | droplet | 3m | 24m | male | Kidney | nan | nan | Spink3 | -14.930218696594238 | 0.0024745729751884937 | 7.925944345288372e-08 | 0.0036662656348198652 | 2.1238898003625066e-116 |
| 18 | facs | 3m | 18m | female | Liver | nan | nan | H2-DMb2 | 14.885748863220215 | 2.0022261182361945e-08 | 0.000606166198849678 | 0.0014720053877681494 | 4.390376186452859e-16 |
| 1 | facs | 18m | 24m | male | BAT | Fat | myeloid cell | Ldhc | 14.876587867736816 | 1.0573597997165507e-08 | 0.0003180994826834649 | 0.000731860229279846 | 1.7215763612974926e-12 |
| 5 | droplet | 3m | 24m | male | Kidney | nan | nan | Calml4 | -14.873334884643555 | 0.0002802493399940431 | 9.33637522848585e-09 | 0.0003948401426896453 | 4.4647651493665767e-128 |
| 19 | facs | 3m | 18m | female | Liver | nan | nan | Ncf4 | 14.87302303314209 | 4.366481398676569e-09 | 0.0001310561056016013 | 0.00029209087369963527 | 1.2853229001718183e-19 |
| 20 | facs | 3m | 18m | female | Liver | nan | nan | Ctsw | 14.871115684509277 | 1.2717961794805888e-08 | 0.0003811567439697683 | 0.0009542901534587145 | 6.471038893091117e-15 |
| 10 | droplet | 3m | 30m | male | nan | nan | nan | Mt4 | -14.864083290100098 | 0.0017565828748047352 | 5.8901502342223466e-08 | 0.005649079568684101 | 0.0 |
| 7 | droplet | 24m | 30m | male | nan | nan | nan | Krt4 | -14.819025039672852 | 0.0012810253538191319 | 4.431767663959363e-08 | 0.00763982767239213 | 2.798121896172275e-226 |
| 6 | droplet | 24m | 30m | male | Kidney | nan | nan | Aldob | 14.767964363098145 | 9.322714333848126e-08 | 0.00260104238986969 | 0.002803180366754532 | 0.0 |
| 21 | facs | 3m | 18m | female | Liver | nan | nan | Il2rb | 14.737149238586426 | 1.0355513779813919e-08 | 0.00028283786377869546 | 0.0009231164585798979 | 2.5425326725069516e-07 |
| 22 | facs | 3m | 18m | female | Liver | nan | nan | Nkg7 | 14.710185050964355 | 3.76863340534328e-08 | 0.0010101880179718137 | 0.002343457890674472 | 5.756067056144295e-18 |
| 23 | facs | 3m | 18m | female | Liver | nan | nan | Cd79b | 14.677618026733398 | 3.478822563351969e-08 | 0.0009116919827647507 | 0.002330661052837968 | 3.5981272982768633e-14 |
| 0 | facs | 3m | 24m | male | Brain_Non-Myeloid | Striatum | oligodendrocyte | Ppil2 | -14.671002388000488 | 9.562943159835413e-05 | 3.664892389565466e-09 | 0.00017788528930395842 | 2.0255371939722143e-15 |
| 24 | facs | 3m | 18m | female | Liver | nan | nan | Ms4a4b | 14.645726203918457 | 1.8090052122943234e-08 | 0.00046373173245228827 | 0.0011971311178058386 | 8.008747700184536e-14 |
| 8 | droplet | 24m | 30m | male | nan | nan | nan | Mt4 | -14.637614250183105 | 0.0015013942029327154 | 5.8901502342223466e-08 | 0.005938917864114046 | 0.0 |
| 11 | droplet | 3m | 30m | male | nan | nan | nan | Krtap3-3 | -14.60097885131836 | 5.209001028561033e-05 | 2.0951480550479573e-09 | 0.0005630051018670201 | 1.30896846649585e-34 |
| 4 | droplet | 3m | 18m | female | nan | nan | nan | Sst | 14.566500663757324 | 1.865071119766526e-08 | 0.00045255591976456344 | 0.007631553336977959 | 3.4318839994188547e-13 |
| 25 | facs | 3m | 18m | female | Liver | nan | nan | Gimap3 | 14.558606147766113 | 2.4720883118334314e-08 | 0.0005965663585811853 | 0.0012602368369698524 | 2.1796109155692484e-22 |
| 0 | droplet | 3m | 18m | male | Kidney | nan | lymphocyte | S100a8 | 14.547069549560547 | 3.248440862080315e-06 | 0.07776413857936859 | 0.07736752927303314 | 2.2933287382995585e-08 |
| 4 | facs | 3m | 24m | male | Liver | Hepatocytes | nan | Cybb | 14.545760154724121 | 6.759796722377587e-09 | 0.00016169928130693734 | 0.00036829643067903817 | 7.240672015713914e-08 |
| 26 | facs | 3m | 18m | female | Liver | nan | nan | Pik3cd | 14.537814140319824 | 2.4443147506048035e-09 | 5.816382690682076e-05 | 0.0001805665815481916 | 2.8055650772211477e-08 |
| 8 | facs | 18m | 24m | male | BAT | nan | nan | Mal | 14.531257629394531 | 3.459501352054417e-09 | 8.193762187147513e-05 | 0.0003025854821316898 | 7.680034329262816e-10 |
| 1 | facs | 3m | 24m | male | Brain_Non-Myeloid | Striatum | oligodendrocyte | Aldh6a1 | -14.523354530334473 | 8.632671233499423e-05 | 3.664892389565466e-09 | 0.00015953877300489694 | 1.2669133046215906e-15 |
| 4 | droplet | 3m | 24m | male | nan | nan | nan | 3110079O15Rik | -14.51693058013916 | 0.0006273901090025902 | 2.6760238469591968e-08 | 0.0018081284360960126 | 0.0 |
| 9 | droplet | 24m | 30m | male | nan | nan | nan | Krt35 | -14.515442848205566 | 3.230738002457656e-05 | 1.3784885455336848e-09 | 0.0004568661388475448 | 3.67697847773925e-31 |
| 27 | facs | 3m | 18m | female | Liver | nan | nan | Ms4a1 | 14.51134967803955 | 1.586522202501328e-08 | 0.0003705313429236412 | 0.0010961487423628569 | 1.1094842261891415e-09 |
| 5 | droplet | 3m | 18m | female | nan | nan | nan | Gcg | 14.504186630249023 | 8.049083533023804e-08 | 0.0018704535905271769 | 0.021522652357816696 | 2.4571826322150386e-32 |
| 2 | facs | 3m | 24m | male | Aorta | Heart | aortic endothelial cell | Igfbp5 | -14.501344680786133 | 0.0008412777679041028 | 3.62733594272413e-08 | 0.0012657494517043233 | 7.773174176474623e-09 |
| 3 | droplet | 3m | 21m | female | nan | nan | nan | Sst | 14.477580070495605 | 1.865071119766526e-08 | 0.00042550486978143454 | 0.007862471044063568 | 2.723753018750103e-19 |
| 0 | facs | 3m | 18m | male | Aorta | Heart | aortic endothelial cell | Igfbp5 | -14.474392890930176 | 0.0008412777679041028 | 3.6957388260816515e-08 | 0.0012657494517043233 | 7.773537900607733e-09 |


In [ ]:
font = {'family' : 'monospace',
        'weight' : 'regular',
        'size'   : 20}

matplotlib.rc('font', **font)

n_plot = 20
fig, ax = plt.subplots(figsize=(12, 12))
ax.barh(np.arange(n_plot), merged_results["dExpression"][:n_plot])
ax.invert_yaxis()
ax.set_xlabel('Expression fold-change')
ax.set_yticks(np.arange(n_plot))
_ = ax.set_yticklabels([
  '{0} {1} {2} {3} {4}'.format(
    row['sex'], 
    str(row['tissue']).rjust(12), 
    str(row['subtissue']).rjust(12), 
    str(row['cell_ontology_class'])[:15].rjust(18), 
    row['gene'].rjust(10)
  )
  for _, row in merged_results.iloc[:n_plot].iterrows()
])